<a href="https://colab.research.google.com/github/anjali-ii/Depression_Detection_from_speech/blob/main/Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Define the directory where your audio files are stored
base_dir = '/content/drive/My Drive/Audio'

In [3]:
pip install pydub

In [4]:
pip install pyworld

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyworld: filename=pyworld-0.3.4-cp310-cp310-linux_x86_64.whl size=865274 sha256=66ef19d55fdbcbe1c41e82488c23b8a01a6e7996d840ea9c46b99c1b6581a1f2
  Stored in directory: /root/.cache/pip/wheels/66/09/8a/a1d79b73d59756f66e9bfe55a199840efc7473adb76ddacdfd
Successfully built pyworld


In [5]:
import os
import numpy as np
from pydub import AudioSegment
import librosa
import pandas as pd
import pyworld as pw
import matplotlib.pyplot as plt
import warnings

In [6]:
# Define parameters for feature extraction
frame_length = 0.050  # Frame length in seconds
hop_length = 0.030    # Hop length (frame shift) in seconds
n_mels = 36           # Number of Mel bands
fs = 16000

In [7]:
# Suppress UserWarning from librosa
warnings.filterwarnings("ignore", message="PySoundFile failed. Trying audioread instead.")

In [8]:
pip install --upgrade librosa

In [9]:
pip install audioread

In [9]:
# Suppress the warning
warnings.filterwarnings('ignore', message='Deprecated as of librosa version 0.10.0.')

In [10]:
# Suppress the FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

In [11]:
mfb_features = []
participant_ids=[]
f0_ap_features =[]
# Iterate over each audio file
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.m4a'):
            # Construct the path to the audio file
            audio_path = os.path.join(root, file)
            #print(f"Processing audio file: {audio_path}")

            # Load audio file
            audio, sr = librosa.load(audio_path, mono=True)

            # Resample to 16 kHz
            target_sr = 16000
            audio_resampled = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)

            frame = audio_resampled.astype(np.double)
            f0, timeaxis = pw.harvest(frame, fs, frame_period=30)
            sp = pw.cheaptrick(frame, f0, timeaxis, fs)
            ap = pw.d4c(frame, f0, timeaxis,fs)
            ap_aggregated = np.mean(ap, axis=1)
            # Calculate Mel filter bank features
            mfb = pw.code_spectral_envelope(sp, fs, n_mels)

            # Ensure all arrays have the same number of frames
            min_frames = min(mfb.shape[0], f0.shape[0], ap_aggregated.shape[0])
            mfb = mfb[:min_frames]
            f0 = f0[:min_frames]
            ap_aggregated = ap_aggregated[:min_frames]

            # Standardization (Mean-variance normalization)
            f0_mean = np.mean(f0)
            f0_std = np.std(f0)
            f0_normalized = (f0 - f0_mean) / f0_std

            ap_aggregated_mean = np.mean(ap_aggregated)
            ap_aggregated_std = np.std(ap_aggregated)
            ap_aggregated_normalized = (ap_aggregated - ap_aggregated_mean) / ap_aggregated_std

            mel_filter_bank_mean = np.mean(mfb, axis=0)
            mel_filter_bank_std = np.std(mfb, axis=0)
            mel_filter_bank_normalized = (mfb - mel_filter_bank_mean) / mel_filter_bank_std

            # Append features to the list
            mfb_features.append(np.concatenate([mel_filter_bank_normalized], axis=1))
            f0_ap_features.append(np.concatenate([f0_normalized[:, np.newaxis], ap_aggregated_normalized[:, np.newaxis]], axis=1))
            # Extract and append severity level
            participant_id = os.path.basename(root)
            participant_ids.extend([participant_id] * min_frames)


In [1]:
# Create DataFrame with F0 and Aperiodicity features
df_f0_ap = pd.DataFrame(np.vstack(f0_ap_features), columns=['F0'] + ['Aperiodicity'])
df_f0_ap['P_id'] = participant_ids

# Convert mfb_features into a single list
mfb_features_concatenated = [item for sublist in mfb_features for item in sublist]

# Create DataFrame with concatenated MFB features and participant IDs
df_mfb = pd.DataFrame({'MFB': mfb_features_concatenated, 'P_id': participant_ids})

# Merge the two DataFrames based on participant IDs
df = pd.merge(df_f0_ap, df_mfb, on='P_id')

# Save merged DataFrame to a CSV file
df.to_csv('frame_features_with_severity.csv', index=False)

NameError: name 'pd' is not defined

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('frame_features_with_severity.csv')

# Display the first few rows of the DataFrame
print(df.tail())


In [ ]:
frame_features = []
participant_ids=[]
# Iterate over each audio file
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.m4a'):
            # Construct the path to the audio file
            audio_path = os.path.join(root, file)
            #print(f"Processing audio file: {audio_path}")

            # Load audio file
            audio, sr = librosa.load(audio_path, mono=True)

            # Resample to 16 kHz
            target_sr = 16000
            audio_resampled = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)

            frame = audio_resampled.astype(np.double)
            f0, timeaxis = pw.harvest(frame, fs, frame_period=30)
            sp = pw.cheaptrick(frame, f0, timeaxis, fs)
            ap = pw.d4c(frame, f0, timeaxis,fs)
            ap_aggregated = np.mean(ap, axis=1)
            # Calculate Mel filter bank features
            mfb = pw.code_spectral_envelope(sp, fs, n_mels)

            # Ensure all arrays have the same number of frames
            min_frames = min(mfb.shape[0], f0.shape[0], ap_aggregated.shape[0])
            mfb = mfb[:min_frames]
            f0 = f0[:min_frames]
            ap_aggregated = ap_aggregated[:min_frames]

            # Standardization (Mean-variance normalization)
            f0_mean = np.mean(f0)
            f0_std = np.std(f0)
            f0_normalized = (f0 - f0_mean) / f0_std

            ap_aggregated_mean = np.mean(ap_aggregated)
            ap_aggregated_std = np.std(ap_aggregated)
            ap_aggregated_normalized = (ap_aggregated - ap_aggregated_mean) / ap_aggregated_std

            mel_filter_bank_mean = np.mean(mfb, axis=0)
            mel_filter_bank_std = np.std(mfb, axis=0)
            mel_filter_bank_normalized = (mfb - mel_filter_bank_mean) / mel_filter_bank_std

            # Append features to the list
            frame_features.append(np.concatenate([mel_filter_bank_normalized, f0_normalized[:, np.newaxis], ap_aggregated_normalized[:, np.newaxis]], axis=1))

            # Extract and append severity level
            participant_id = os.path.basename(root)
            participant_ids.extend([participant_id] * min_frames)

# Concatenate features and severity levels into a single DataFrame
df1 = pd.DataFrame(np.vstack(frame_features), columns=[f'MFB_{i}' for i in range(n_mels)] + ['F0'] + ['Aperiodicity'])
df1['P_id'] = participant_ids

# Save DataFrame to a CSV file
df1.to_csv('frame_features1_with_severity.csv', index=False)

# Load the CSV file into a DataFrame
df1 = pd.read_csv('frame_features1_with_severity.csv')

# Display the first few rows of the DataFrame
print(df1.tail())

In [ ]:
total_rows = len(df)
total_rows

75

In [ ]:
# Load severity scores from Excel sheet
file_path = '/content/drive/My Drive/PHQ.xlsx'
severity_scores_df = pd.read_excel(file_path)

In [ ]:
# Merge feature DataFrame with severity scores DataFrame based on participant ID
merged_df = pd.merge(df, severity_scores_df, on='P_id', how='inner')

KeyError: 'P_id'

In [ ]:
merged_df[56000:]

In [ ]:
# Extract features (X) and labels (Y)
X = merged_df.drop(['P_id', 'Score', 'Category'], axis=1)  # Drop columns for ID, score, and category to get features
Y = merged_df[['Score', 'Category']]  # Get columns for score and category as labels

In [ ]:
print("Unique values in the 'Category' column:", merged_df['Category'].unique())

In [ ]:
# Display the shapes of X and Y
print("Shape of features (X):", X.shape)
print("Shape of labels (Y):", Y.shape)

In [ ]:
# Ensure that the feature vectors are reshaped appropriately for input into the CNN
# The shape should be (num_samples, num_features, num_channels) for a single-channel CNN
# If you have multiple channels (e.g., for different types of features), you'll need to adjust accordingly
num_samples = X.shape[0]  # Number of samples
num_features = X.shape[1]  # Number of features
num_channels = 1  # Since it's a single-channel CNN

In [ ]:
# Convert X to a NumPy array
X = X.values

# Reshape the feature vectors
X = X.reshape(num_samples, num_features, num_channels)

# Print the shapes of X and y to verify
print("Shape of feature vectors (X):", X.shape)
print("Shape of labels (Y):", Y.shape)

In [ ]:
# Extract feature columns
features = merged_df.columns[:-1]  # Exclude the 'Severity' column

# Plot bar graphs for each feature
for feature in features:
    if 'MFB_' in feature:  # Check if the feature is Mel Filter Bank
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_title(f'Bar Graph of {feature}')
        ax.set_xlabel('Category')
        ax.set_ylabel('Mean Value')
        merged_df.groupby('Category')[feature].mean().plot(kind='bar', ax=ax)
        plt.tight_layout()
        plt.show()

In [ ]:
# Extract feature columns
features = merged_df.columns[:-1]  # Exclude the 'Severity' column

# Plot bar graphs for each feature
for feature in features:
    if 'F0' in feature:  # Check if the feature is Mel Filter Bank
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_title(f'Bar Graph of {feature}')
        ax.set_xlabel('Severity')
        ax.set_ylabel('Mean Value')
        merged_df.groupby('Category')[feature].mean().plot(kind='bar', ax=ax)
        plt.tight_layout()
        plt.show()

In [ ]:
# Extract feature columns
features = merged_df.columns[:-1]  # Exclude the 'Severity' column

# Plot bar graphs for each feature
for feature in features:
    if 'Aperiodicity' in feature:  # Check if the feature is Mel Filter Bank
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_title(f'Bar Graph of {feature}')
        ax.set_xlabel('Severity')
        ax.set_ylabel('Mean Value')
        merged_df.groupby('Category')[feature].mean().plot(kind='bar', ax=ax)
        plt.tight_layout()
        plt.show()

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class MultiTaskCNN(nn.Module):
    def __init__(self, num_mels=38):
        super(MultiTaskCNN, self).__init__()
        # First convolutional layer with three different kernels
        self.conv1_1 = nn.Conv1d(in_channels=num_mels, out_channels=50, kernel_size=3, stride=3)
        self.conv1_2 = nn.Conv1d(in_channels=num_mels, out_channels=50, kernel_size=10, stride=3)
        self.conv1_3 = nn.Conv1d(in_channels=num_mels, out_channels=50, kernel_size=15, stride=3)

        # Second convolutional layer
        self.conv2 = nn.Conv1d(in_channels=150, out_channels=50, kernel_size=3, stride=3)

        # Fully connected layer
        self.fc = nn.Linear(50 * 4, 150)  # Assuming input size of 4

        # Output layers
        self.fc_regression = nn.Linear(150, 1)  # Regression output
        self.fc_classification = nn.Linear(150, 5)  # Classification output

    def forward(self, x):
        # First convolutional layer
        x1 = nn.functional.relu(self.conv1_1(x))
        x2 = nn.functional.relu(self.conv1_2(x))
        x3 = nn.functional.relu(self.conv1_3(x))
        x = torch.cat((x1, x2, x3), dim=1)  # Concatenate along channel dimension

        # Second convolutional layer
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool1d(x, kernel_size=2)

        # Print the shape of the tensor before flattening
        print("Shape before flattening:", x.shape)

        # Flatten the output
        x = x.view(x.size(0), -1)

        # Fully connected layer
        x = nn.functional.relu(self.fc(x))

        # Output layers
        regression_output = self.fc_regression(x)  # Regression output
        classification_output = self.fc_classification(x)  # Classification output

        return regression_output, classification_output

# Create an instance of the MultiTaskCNN model
model = MultiTaskCNN()
print(model)

In [ ]:
from sklearn.model_selection import train_test_split
Y=Y.values
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
import torch.optim as optim

# Define loss functions for regression and classification
criterion_regression = nn.MSELoss()
criterion_classification = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Create DataLoader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss_regression = 0.0
    train_loss_classification = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()  # Clear gradients
        regression_output, classification_output = model(inputs)  # Forward pass
        loss_regression = criterion_regression(regression_output, targets[:, 0])  # Calculate regression loss
        loss_classification = criterion_classification(classification_output, targets[:, 1].long())  # Calculate classification loss
        loss = loss_regression + loss_classification  # Total loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        train_loss_regression += loss_regression.item() * inputs.size(0)
        train_loss_classification += loss_classification.item() * inputs.size(0)

    # Validation
    model.eval()
    val_loss_regression = 0.0
    val_loss_classification = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            regression_output, classification_output = model(inputs)  # Forward pass
            loss_regression = criterion_regression(regression_output, targets[:, 0])  # Calculate regression loss
            loss_classification = criterion_classification(classification_output, targets[:, 1].long())  # Calculate classification loss
            val_loss_regression += loss_regression.item() * inputs.size(0)
            val_loss_classification += loss_classification.item() * inputs.size(0)

    # Average losses
    train_loss_regression /= len(train_loader.dataset)
    train_loss_classification /= len(train_loader.dataset)
    val_loss_regression /= len(val_loader.dataset)
    val_loss_classification /= len(val_loader.dataset)

    # Print epoch statistics
    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss (Regression): {train_loss_regression:.4f}, '
          f'Train Loss (Classification): {train_loss_classification:.4f}, '
          f'Val Loss (Regression): {val_loss_regression:.4f}, '
          f'Val Loss (Classification): {val_loss_classification:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'multi_task_cnn.pth')
